In [121]:
import os
import sys
from logging import DEBUG
from pathlib import Path

from dotenv import load_dotenv

from yfpy import Data
from yfpy.logger import get_logger
from yfpy.query import YahooFantasySportsQuery

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json

[2023-12-18 17:21:37,030 DEBUG] [matplotlib.__init__.wrapper] CACHEDIR=C:\Users\mikem\.matplotlib
[2023-12-18 17:21:37,037 DEBUG] [matplotlib.font_manager.font_manager._load_fontmanager] Using fontManager instance from C:\Users\mikem\.matplotlib\fontlist-v330.json
[2023-12-18 17:21:37,037 DEBUG] [matplotlib.font_manager.font_manager._load_fontmanager] Using fontManager instance from C:\Users\mikem\.matplotlib\fontlist-v330.json


### Environment variables

In [102]:
project_dir = os.path.abspath('')

In [103]:
sys.path.insert(0, str(project_dir))

In [104]:
load_dotenv(dotenv_path=f"{project_dir}/auth/.env")

True

### Set directories

In [105]:
auth_dir = f"{project_dir}/auth"

In [106]:
data_dir = f"{project_dir}/output"

### Regenerate functions

In [107]:
def write_league_standings(game_keys, league_ids):
    for year, id in zip(game_keys['season'], game_keys['code']):
        print(f"Attempting to write year {year}")
        yahoo_query = YahooFantasySportsQuery(
                auth_dir,
                league_id=league_ids[str(year)],
                game_code='nfl',
                game_id=id
            )
        try:
            data.save(file_name=f"standings_year_{year}", yf_query=yahoo_query.get_league_standings)
        except:
            print(f"Could not save data for year: {year}, code: {id}")
        else:
            pass

### Load data

In [108]:
data = Data(data_dir)

In [109]:
game_keys = data.load("game keys")

In [110]:
game_keys_list = []
for game in game_keys:
    entry = {'season': game.season, 'code': game.game_id}
    game_keys_list.append(entry)

In [111]:
game_keys_df = pd.DataFrame(game_keys_list)

In [112]:
game_keys_df = game_keys_df[game_keys_df['season'] > 2014]

In [113]:
game_keys_df

,season,code
16,2015,348
17,2016,359
18,2017,371
19,2018,380
20,2019,390
21,2020,399
22,2021,406
23,2022,414
24,2023,423


In [114]:
with open('output/league_ids.json') as f:
    league_ids = json.load(f)

In [115]:
league_ids

{'2015': 853925,
 '2016': 810238,
 '2017': 192551,
 '2018': 799346,
 '2019': 820083,
 '2020': 394682,
 '2021': 364325,
 '2022': 564704,
 '2023': 333146}

In [116]:
# Uncomment to regenerate standings per modern era year if needed
write_league_standings(game_keys_df, league_ids)

Attempting to write year 2015
Attempting to write year 2016
Attempting to write year 2017
Attempting to write year 2018
Attempting to write year 2019
Attempting to write year 2020
Attempting to write year 2021
Attempting to write year 2022
Attempting to write year 2023


### Generate fresh standings dataframe

In [88]:
standings_filenames = [filename.replace('.json','') for filename in os.listdir('./output/standings_by_season/')]

In [117]:
all_team_stats = []
for file in standings_filenames:
    working_file = data.load(f'standings_by_season/{file}')
    for i in working_file['teams']:
        team_info = i['team'].serialized()
        team_info['season'] = file.replace('standings_year_','')
        all_team_stats.append(team_info)

In [118]:
standings_to_date_df = pd.json_normalize(all_team_stats)

In [119]:
standings_to_date_df.to_csv('output/standings_to_date.csv', index=False)

In [120]:
df = standings_to_date_df.copy()

In [128]:
mask1 = df['season'] == '2022'

In [135]:
df22 = df[mask1] 
df22

,clinched_playoffs,division_id,draft_grade,draft_position,draft_recap_url,faab_balance,has_draft_grade,league_scoring_type,name,number_of_moves,...,team_standings.outcome_totals.percentage,team_standings.outcome_totals.ties,team_standings.outcome_totals.wins,team_standings.playoff_seed,team_standings.points_against,team_standings.points_for,team_standings.rank,team_standings.streak.type,team_standings.streak.value,is_owned_by_current_login
0,1.0,2.0,C+,14.0,https://football.fantasysports.yahoo.com/2015/...,12,1,head,b'Starr is Born',24,...,0.692,0,9,3.0,1160.91,1302.03,1,win,5,NaN
1,1.0,1.0,C,7.0,https://football.fantasysports.yahoo.com/2015/...,0,1,head,b'Thug Bungalow',28,...,0.615,0,8,4.0,1273.94,1355.37,2,win,2,1.0
2,1.0,1.0,B,12.0,https://football.fantasysports.yahoo.com/2015/...,0,1,head,b'The Ice Cold Bruschi',30,...,0.769,0,10,1.0,1225.08,1457.66,3,win,1,NaN
3,1.0,2.0,C+,9.0,https://football.fantasysports.yahoo.com/2015/...,6,1,head,b'who invited canada',46,...,0.769,0,10,2.0,1234.93,1425.64,4,win,2,NaN
4,1.0,2.0,B-,6.0,https://football.fantasysports.yahoo.com/2015/...,60,1,head,b'The fishingtons',23,...,0.615,0,8,5.0,1103.02,1302.04,5,loss,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,NaN,NaN,D-,NaN,https://football.fantasysports.yahoo.com/f1/33...,5,1,head,b'who invited canada',21,...,0.429,0,6,10.0,1594.64,1544.62,10,win,1,NaN
122,NaN,NaN,C,NaN,https://football.fantasysports.yahoo.com/f1/33...,100,1,head,"b""Jadeveon's Hit Men""",5,...,0.429,0,6,11.0,1636.64,1530.28,11,loss,2,NaN
123,NaN,NaN,A+,NaN,https://football.fantasysports.yahoo.com/f1/33...,100,1,head,b'Wereback',6,...,0.357,0,5,12.0,1442.22,1433.82,12,loss,1,NaN
124,NaN,NaN,B-,NaN,https://football.fantasysports.yahoo.com/f1/33...,21,1,head,b'The fishingtons',9,...,0.286,0,4,NaN,1588.12,1422.80,13,win,1,NaN
